In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [0]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [0]:
from google.colab import drive
drive.mount('drive')

In [0]:
votes = pd.read_csv("drive/My Drive/sna9/train3_data.csv")

In [0]:
votes.head(2)

,userId,itemId,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,acc,se_Xmin,se_Ymin,se_Xmax,se_Ymax,Y_xmin,Y_ymin,Y_xmax,Y_ymax,L,H,P,S,L/H,D,iou,best
0,1164,6132,62,656,276,803,47,650,213,812,0.988892,225,36,3969,81,1,1,0,1,214.0,147.0,722.0,31458.0,1.455782,259.624729,0.613974,1
1,1399,6132,184,662,235,740,47,650,213,812,0.936663,18769,144,484,5184,0,0,1,0,51.0,78.0,258.0,3978.0,0.653846,93.193347,0.079069,0


# select good users

In [5]:
# by max iou
limit = 0.475
goods = votes[votes.iou > limit]
goods.shape # (1693, 27) not enough...

(2678, 27)

In [26]:
# and bads
bads = votes[votes.iou < limit]
bads.shape # (1693, 27) not enough...

(2994, 27)

In [0]:
len(goods.userId.unique()) # 797 from 1355
#len(votes.userId.unique())

996

In [0]:
def func(df):
    good1 = df.groupby("itemId")['Xmin', 'Ymin'].min()
    good2 = df.groupby("itemId")['Xmax', 'Ymax', 'Xmin_true',	'Ymin_true',	'Xmax_true',	'Ymax_true'].max()
    good = pd.concat([good1, good2], axis=1)
    good['iou'] = good.apply(intersection_over_union, axis=1)
    print(good['iou'].mean())

In [59]:
votest[['L','H']].describe()

,L,H
count,3603.000000,3603.000000
mean,311.626145,259.190397
std,224.988392,183.753729
min,1.000000,1.000000
25%,124.000000,105.000000
50%,269.000000,226.000000
75%,461.000000,387.500000
max,995.000000,1006.000000


In [16]:
# говнюки
for i in [0.1, 0.15, 0.2, 0.25]: #0.3, 0.35, 0.4, 0.45, 0.475]:
    shiters = votes[votes.iou < i]
    print(i)
    shiters.iou.mean()
    shiters.head()
    func(votes.drop(shiters.index))
    func(votes)
    print(votes.drop(shiters.index).shape)
    print(len(votes.drop(shiters.index).itemId.unique()))

0.1
0.5821864175273562
0.5822325332074827
(5006, 27)
941
0.15
0.5822608372490312
0.5822325332074827
(4863, 27)
941
0.2
0.5830294390080151
0.5822325332074827
(4699, 27)
939
0.25
0.5837408527107728
0.5822325332074827
(4483, 27)
939


In [17]:
# сравню с тестом
for i in [0.1, 0.15, 0.2, 0.25]:
    shiters = votes[votes.iou < i]
    print(i)
    uList = shiters.userId
    v = votest[votest.userId.isin(uList)]
    print(votest.drop(v.index).shape)
    print(len(votest.drop(v.index).itemId.unique()))

0.1
(2656, 12)
622
0.15
(2404, 12)
618
0.2
(2203, 12)
608
0.25
(1905, 12)
584


In [0]:
# ok. Get for iou < 0.2:
shiters = votes[votes.iou < 0.2]
v = votest[votest.userId.isin(shiters.userId)]
tgoods = votest.drop(v.index)

tgood1 = tgoods.groupby("itemId")['Xmin', 'Ymin'].min()
tgood2 = tgoods.groupby("itemId")['Xmax', 'Ymax'].max()
tgood = pd.concat([tgood1, tgood2], axis=1)

diff = list(set(maxbox.itemId.unique()) - set(tgood.index.unique()))
add = maxbox[maxbox.itemId.isin(diff)].set_index('itemId')
tgood = pd.concat([tgood, add]).sort_index()

tgood.to_csv("drive/My Drive/sna9/shiters020.csv", header=None)

In [28]:
func(goods) # 0.6388167729826996 not bad...
func(bads)  # 0.4583323473987864    

0.6321808729675101
0.39753920903896556


In [30]:
func0(goods) # 0.6388167729826996 not bad...
func0(bads)  # 0.4583323473987864    

0.6232133942014288
0.40350039635444335


In [0]:
(0.6388167729826996 + 0.4583323473987864)/2 

0.548574560190743

а по средним?

In [0]:
def func1(df):
    good = df.groupby("itemId")['Xmin', 'Ymin', 'Xmax', 'Ymax', 'Xmin_true',	'Ymin_true',	'Xmax_true',	'Ymax_true'].mean()
    good['iou'] = good.apply(intersection_over_union, axis=1)
    print(good['iou'].mean())

In [15]:
func1(goods) # 0.6331308771672398 not bad...
func1(bads) # 0.3727520783610892

0.774163147592442
0.4783154102567855


...разобью вначале на левые нижние и правые верхние, потом на гуд/бэд

In [0]:
votes[['Xmin', 'Ymin']].describe() # mean	118.828008	654.520301
votes[['Xmax', 'Ymax']].describe() # mean	428.742481	922.089474

вообще, даже на 5 категорий -


1.   левые нижние - все коорд мин
2.   левые верхние - иксы маленькие, игреки большие
3.   правые верхние - все коорд макс
4. правые нижние - иксы большие, игреки мал
5. центральные - мин х и у большие, а макс - маленькие

ЧЕРТ!!! куда-то пропадают индексы...
надо не пять, а девять - еще крестом



In [0]:
xmean, ymean, Xmean, Ymean = 118.828008, 654.520301, 428.742481, 922.089474
def zones(votes):
    center = votes[(votes.Xmin > xmean) & (votes.Ymin > ymean) & (votes.Xmax < Xmean) & (votes.Ymax < Ymean)]
    print(center.shape) # (77, 27) не стоит выделки
    centerL = votes[(votes.Xmin < xmean) & (votes.Ymin > ymean) & (votes.Ymax < Ymean)]
    print(centerL.shape)  # (475, 27)
    centerR = votes[(votes.Xmin > Xmean) & (votes.Ymin > ymean) & (votes.Ymax < Ymean)]
    print(centerR.shape)  # (14, 27) ???
    centerU = votes[(votes.Xmin > xmean) & (votes.Xmax < Xmean) & (votes.Ymax > Ymean)]
    print(centerU.shape)  # (261, 27)
    centerD = votes[(votes.Xmin > xmean) & (votes.Xmax < Xmean) & (votes.Ymax < ymean)]
    print(centerD.shape)  # (99, 27)

    # corners
    cornerLD = votes[(votes.Xmax < xmean) & (votes.Ymax < ymean)]
    print(cornerLD.shape) # (269, 27)
    cornerLU = votes[(votes.Xmax < xmean) & (votes.Ymax > Ymean)]
    print(cornerLU.shape) # (141, 27)
    cornerRU = votes[(votes.Xmin > Xmean) & (votes.Ymin > Ymean)]
    print(cornerRU.shape) # (17, 27)
    cornerRD = votes[(votes.Xmin > Xmean) & (votes.Ymax < ymean)]
    print(cornerRD.shape) # (11, 27)
    
    # КРУПНЫЕ ЗОНЫ крестом
    LD = votes[(votes.Xmin < xmean) & (votes.Ymin < ymean)]
    print(LD.shape) # (1838, 27)
    LU = votes[(votes.Xmin < xmean) & (votes.Ymax > Ymean)]
    print(LU.shape) # (1485, 27)
    RD = votes[(votes.Xmax > Xmean) & (votes.Ymin < ymean)]
    print(RD.shape) # (1456, 27)
    RU = votes[(votes.Xmax > Xmean) & (votes.Ymax > Ymean)]
    print(RU.shape) # (1512, 27)
    return centerR, cornerRD, cornerRU, center, centerD, centerU, cornerLD, centerL, cornerLU, LD, RD, LU, RU

In [0]:
14, 11, 17, 77, 99, 261, 269, 475, 141, 1838, 1456, 1485, 1512
[centerR, cornerRD, cornerRU, center, centerD, centerU, cornerLD, centerL, cornerLU, LD, RD, LU, RU]

In [8]:
# КРУПНЫЕ ЗОНЫ крестом
LD = votes[(votes.Xmin < xmean) & (votes.Ymin < ymean)]
LD.shape # (1838, 27)
LU = votes[(votes.Xmin < xmean) & (votes.Ymax > Ymean)]
LU.shape # (1485, 27)
RD = votes[(votes.Xmax > Xmean) & (votes.Ymin < ymean)]
RD.shape # (1456, 27)
RU = votes[(votes.Xmax > Xmean) & (votes.Ymax > Ymean)]
RU.shape # (1512, 27)

(1512, 27)

Вот имеем иерархию точности по зонам, начиная с САМОЙ ТОЧНОЙ:


1.   ПРАВЫЙ ВЕРХНИЙ угол
2.   ЛЕВЫЙ ВЕРХНИЙ угол
3. ПРАВЫЙ нижНИЙ угол
4. леВЫЙ нижНИЙ угол

In [0]:
func(LD) # 0.5727845738186389
func(LU)  # 0.5807225102761434   
func(RD) # 0.5754962380687619
func(RU)  # 0.5833685239864976   

func(cornerLD) # 0.3781851353904525
func(cornerLU)  # 0.5619822246246656  
func(cornerRD) # 0.05633007885052714
func(cornerRU)  # 0.07652515835570738

func(center) # 0.19687072884338763
func(centerL)  # 0.5149010177868754  
func(centerD) # 0.2757125596426167
func(centerU) # 0.3523829206507815
func(centerR) # 0.012478350846567579

In [0]:
# общее дерьмо
df = pd.concat([LD, LU, RD, RU, cornerLD, cornerLU, cornerRD, cornerRU, center, centerL, centerR, centerU, centerD]).set_index('itemId')
df.shape (7655, 26)

In [25]:
func(df) 0.5819580095102326

0.5819580095102326


Ожидаемо. Надо в цикле проверить уменьщение df кусками, начиная с худших:


In [0]:
rating = [centerR, cornerRD, cornerRU, center, centerD, centerU, cornerLD, centerL, cornerLU, LD, RD, LU, RU]

In [28]:
for i in range(13):
    rate = rating[i:]
    df0 = pd.concat(rate)
    print(len(df0.itemId.unique()))
    func(df0)

940
0.5819580095102326
940
0.5819146295476925
940
0.5819146295476925
940
0.5819146295476925
937
0.5818398428514141
934
0.5823674492290505
928
0.5825791735648692
928
0.5825791735648692
854
0.5811122669413018
854
0.5811122669413018
659
0.583574911412011
487
0.58267383921282
336
0.5833685239864976


## Результаты крайне интересные.


1.   Некоторые (centerR, cornerRD, cornerRU, center, centerD) - на рез-т ваще не влияют
2.   Хотя точность и лучшая на RD, LU, RU - пропадают индексы, надо экстраполировать (уже столкнулся). Причем можно экстраполировать именно на основе зон - т.е. fillna средними или соседними
3. На тесте может быть две ситуации - разбиение по зонам на основе тренинговых лимитов, и на основе лимитов теста



In [12]:
votest = pd.read_csv("drive/My Drive/sna9/test1_data.csv", index_col=0)
votest.head()

,userId,itemId,Xmin,Ymin,Xmax,Ymax,L,H,P,S,L/H,D
0,1581,34804,86,640,242,743,156,103,518,16068,1.514563,186.935818
1,1351,34804,85,655,273,766,188,111,598,20868,1.693694,218.323155
2,161,34804,85,648,268,748,183,100,566,18300,1.830000,208.540164
3,313,5704,32,618,647,1268,615,650,2530,399750,0.946154,894.832387
4,1097,5704,25,620,602,1242,577,622,2398,358894,0.927653,848.417939


In [42]:
shiters = votest[(votest.L<15) | (votest.H<15)]
print(votest.drop(shiters.index).shape)
print(len(votest.drop(shiters.index).itemId.unique()))


(3443, 12)
618


In [0]:
rate_test = [centerU, cornerLD, centerL, cornerLU, LD, RD, LU, RU]

In [0]:
for i in range(10):
    rate = dfs[i:]
    df0 = pd.concat(rate)
    tgood1 = df0.groupby("itemId")['Xmin', 'Ymin'].min()
    tgood2 = df0.groupby("itemId")['Xmax', 'Ymax'].max()
    tgood = pd.concat([tgood1, tgood2], axis=1)
    diff = list(set(maxbox.itemId.unique()) - set(tgood.index.unique()))
    add = maxbox[maxbox.itemId.isin(diff)].set_index('itemId')
    tgood = pd.concat([tgood, add]).sort_index()
    tgood.to_csv("drive/My Drive/sna9/" + str(i*2) +".csv", header=None)

In [0]:
df0

In [0]:
dfs = zones(votest)

(54, 12)
(410, 12)
(8, 12)
(113, 12)
(70, 12)
(133, 12)
(72, 12)
(10, 12)
(16, 12)
(1276, 12)
(976, 12)
(1054, 12)
(917, 12)

(47, 12)
(406, 12)
(8, 12)
(142, 12)
(75, 12)
(111, 12)
(67, 12)
(12, 12)
(16, 12)
(1211, 12)
(995, 12)
(1034, 12)
(965, 12)

In [0]:
# ?????????????
votest[['Xmin', 'Ymin']].describe() # mean	113.690258	643.97058
votest[['Xmax', 'Ymax']].describe() # mean	425.316403	903.160977

In [68]:
xmean, ymean, Xmean, Ymean #= 113.690258, 643.97058,	425.316403,	903.160977

(113.690258, 643.97058, 425.316403, 903.160977)

In [38]:
LD = votest[(votest.Xmin < xmean) & (votest.Ymin < ymean)]
LD.shape # (1211, 12)
LU = votest[(votest.Xmin < xmean) & (votest.Ymax > Ymean)]
LU.shape # (995, 12)
RD = votest[(votest.Xmax > Xmean) & (votest.Ymin < ymean)]
RD.shape # (1034, 12)
RU = votest[(votest.Xmax > Xmean) & (votest.Ymax > Ymean)]
RU.shape # (965, 12)

(965, 12)

In [39]:
# общее дерьмо
df = pd.concat([LU, RD, RU])
df.shape

(2994, 12)

In [91]:
df.tail()

,userId,itemId,Xmin,Ymin,Xmax,Ymax,L,H,P,S,L/H,D
3605,1046,13921,125,838,507,1082,382,244,1252,93208,1.565574,453.276957
3606,1170,13921,96,806,488,1166,392,360,1504,141120,1.088889,532.225516
3612,1361,33300,463,391,969,1258,506,867,2746,438702,0.583622,1003.855069
3613,656,33300,480,461,971,1159,491,698,2378,342718,0.703438,853.396157
3614,1513,33300,387,432,992,1182,605,750,2710,453750,0.806667,963.600021


In [42]:
len(votest.itemId.unique())
len(df.itemId.unique())

435

In [40]:
tgood1 = df.groupby("itemId")['Xmin', 'Ymin'].min()
tgood2 = df.groupby("itemId")['Xmax', 'Ymax'].max()
tgood = pd.concat([tgood1, tgood2], axis=1)
tgood.head()

,Xmin,Ymin,Xmax,Ymax
itemId,,,,
18,126,634,764,1132
62,35,824,639,1262
146,17,211,643,1118
186,82,890,1028,1544
215,228,639,828,1038


In [41]:
tgood.shape

(435, 4)

In [0]:
tgoods = votest[votest.userId.isin(goods.userId.unique())]

In [18]:
tgoods.shape # (2803, 13) at iou > 0.5

(1560, 13)

In [19]:
tgood1 = tgoods.groupby("itemId")['Xmin', 'Ymin'].min()
tgood2 = tgoods.groupby("itemId")['Xmax', 'Ymax'].max()
tgood = pd.concat([tgood1, tgood2], axis=1)
tgood.head()

,Xmin,Ymin,Xmax,Ymax
itemId,,,,
18,126,774,694,1010
19,28,559,113,721
33,21,380,414,652
62,35,824,639,1262
114,41,575,216,780


In [65]:
tgood.shape # (626, 4)

(568, 4)

In [18]:
# надо дополнить:
maxbox = pd.read_csv("drive/My Drive/sna9/maxbox.csv", header=None).rename(columns={0: "itemId", 1: "Xmin", 2:'Ymin', 3:'Xmax', 4:'Ymax'})
maxbox.head()

,itemId,Xmin,Ymin,Xmax,Ymax
0,18,126,634,764,1132
1,19,27,559,119,729
2,33,21,380,414,652
3,62,35,824,639,1262
4,114,41,570,216,780


In [0]:
diff = list(set(maxbox.itemId.unique()) - set(tgood.index.unique()))
add = maxbox[maxbox.itemId.isin(diff)].set_index('itemId')
tgood = pd.concat([tgood, add]).sort_index()

In [0]:
tgood.head()

,Xmin,Ymin,Xmax,Ymax
itemId,,,,
18,126,634,764,1132
19,27,559,119,729
33,21,380,414,652
62,35,824,639,1262
114,41,575,216,780


In [0]:
tgood.to_csv("drive/My Drive/sna9/corners0.csv", header=None)

# END

In [0]:
#users_Xmin,  users_Ymin, users_Xmax, users_Ymax = [], [], [], []
u1_xmin, u2_xmin = [], []
uList2 = users('se_Xmin', mse1, u1_xmin, u2_xmin)[1]
#good_Xmin1 = votes[votes.userId.isin(uList1)]
good_Xmin1 = votes.drop(uList2)
print(good_Xmin1.shape)    # (1017, 25)          другое дело! 

(1017, 25)


In [0]:
len(good_Xmin1.userId.unique())

562

In [0]:
good_Xmin1.head(20)

In [0]:
u1_ymin, u2_ymin = [], []
uList2 = users('se_Ymin', mse2, u1_ymin, u2_ymin)[1]
good_Xmin2 = votes.drop(uList2)
print(good_Xmin2.shape)    # (1060, 25)        

u1_xmax, u2_xmax = [], []
uList3 = users('se_Xmax', mse3, u1_xmax, u2_xmax)[1]
good_Xmin3 = votes.drop(uList3)
print(good_Xmin3.shape)    # (1060, 25)      

u1_ymax, u2_ymax = [], []
uList4 = users('se_Ymax', mse4, u1_ymax, u2_ymax)[1]
good_Xmin4 = votes.drop(uList4)
print(good_Xmin4.shape)    # (1058, 25)             

(1060, 25)
(1060, 25)
(1058, 25)


Так. Я МОГУ по пунктам:
1. Присвоить этим гудам 1, остальным 0 - и, как раньше, посчитать xgb.
2. Сразу взять из теста этих гудусеров и заполнить Xmin  ими. Недостающие поля - из median
3. Натренить xgb на лучших усерах по каждой переменной, затем взять медианные значения теста и предсказать по ним

Начну со второго  варианта, причем загружу лучший вариант:

In [0]:
answers = pd.read_csv("drive/My Drive/sna9/train_answers.csv")
best = pd.read_csv("drive/My Drive/sna9/xgb_more3.csv", header=None).rename(columns={0: "itemId", 1: "Xmin", 2:'Ymin', 3:'Xmax', 4:'Ymax'})
votest = pd.read_csv("drive/My Drive/sna9/test1_data.csv", index_col=0)

In [0]:
answers.head(3)

In [0]:
answers.shape

(943, 5)

In [0]:
best.head(3)

,itemId,Xmin,Ymin,Xmax,Ymax
0,18,144.0,634.0,764.0,1132.0
1,19,28.0,559.0,113.0,727.0
2,33,43.0,388.0,365.0,629.0


In [0]:
best.shape

(630, 5)

In [0]:
# votest.shape # (3603, 12)
votest.head(2)

,userId,itemId,Xmin,Ymin,Xmax,Ymax,L,H,P,S,L/H,D
0,1581,34804,86,640,242,743,156,103,518,16068,1.514563,186.935818
1,1351,34804,85,655,273,766,188,111,598,20868,1.693694,218.323155


In [0]:
# ИТАК. Выберу лучшие Xmin через лучших юзеров:
good_Xmin1 = votest[votest.userId.isin(uList1)]
print(good_Xmin1.shape) # (2789, 12)

good_Ymin1 = votest[votest.userId.isin(uList2)]
print(good_Ymin1.shape) # (2809, 12)

good_Xmax1 = votest[votest.userId.isin(uList3)]
print(good_Xmax1.shape) # (2836, 12)

good_Ymax1 = votest[votest.userId.isin(uList4)]
print(good_Ymax1.shape) # (2775, 12)

(2061, 12)
(2104, 12)
(2207, 12)
(2214, 12)


Хорошие предсказания по каждой переменной:

In [0]:
quo_test1 = good_Xmin1.groupby("itemId")[['Xmin']].median().reset_index()
quo_test1.shape # (626, 2)

(618, 2)

In [0]:
diff = best[best.itemId.isin(set(best.itemId) - set(quo_test1.itemId))].drop(['Ymin','Xmax','Ymax'], axis=1)
quo_test1 = pd.concat([quo_test1, diff]).set_index('itemId')
quo_test1.shape

(630, 1)

In [0]:
quo_test1.head()

In [0]:
quo_test2 = good_Ymin1.groupby("itemId")[['Ymin']].median().reset_index()
quo_test2.shape # (629, 2)

(617, 2)

In [0]:
diff = best[best.itemId.isin(set(best.itemId) - set(quo_test2.itemId))].drop(['Xmin','Xmax','Ymax'], axis=1)
quo_test2 = pd.concat([quo_test2, diff]).set_index('itemId')
quo_test2.shape

(630, 1)

In [0]:
quo_test2.head()

In [0]:
quo_test3 = good_Xmax1.groupby("itemId")[['Xmax']].median().reset_index()
quo_test3.shape # (625, 2)

(620, 2)

In [0]:
diff = best[best.itemId.isin(set(best.itemId) - set(quo_test3.itemId))].drop(['Xmin','Ymin','Ymax'], axis=1)
quo_test3 = pd.concat([quo_test3, diff]).set_index('itemId')
quo_test3.shape

(630, 1)

In [0]:
quo_test4 = good_Ymax1.groupby("itemId")[['Ymax']].median().reset_index()
quo_test4.shape # (625, 2)

(621, 2)

In [0]:
diff = best[best.itemId.isin(set(best.itemId) - set(quo_test4.itemId))].drop(['Xmin','Ymin','Xmax'], axis=1)
quo_test4 = pd.concat([quo_test4, diff]).set_index('itemId')
quo_test4.shape

(630, 1)

In [0]:
# Пора объединяться....
quo_test = pd.concat([quo_test1, quo_test2, quo_test3, quo_test4], axis=1)
quo_test.head()

,Xmin,Ymin,Xmax,Ymax
itemId,,,,
18,135.0,704.0,694.0,1071.0
19,28.0,559.0,116.0,727.0
33,43.0,388.0,365.0,629.0
62,101.5,980.0,623.0,1163.0
114,69.0,577.5,206.5,769.0


In [0]:
quo_test.sort_index().to_csv("drive/My Drive/sna9/best_diff1.csv", header=None)

0.5179808  fuck


# XGB from best

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [0]:
good_Xmin1.head(2)

,userId,itemId,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,acc,se_Xmin,se_Ymin,se_Xmax,se_Ymax,Y_xmin,Y_ymin,Y_xmax,Y_ymax,L,H,P,S,L/H,D
5,1092,6132,39,649,249,825,47,650,213,812,0.996058,64,1,1296,169,1,1,1,1,210.0,176.0,772.0,36960.0,1.193182,274.00000
7,1508,31297,25,399,120,485,18,390,122,472,0.997822,49,81,4,169,1,0,1,0,95.0,86.0,362.0,8170.0,1.104651,128.14445


In [0]:
Xmin_X = good_Xmin1[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']]
Xmin_Y = good_Xmin1.Xmin_true

In [0]:
clf = GradientBoostingRegressor().fit(Xmin_X, Xmin_Y )  
clf.score(Xmin_X, Xmin_Y)

0.3083456627990352

In [0]:
def xgb(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    clf = GradientBoostingRegressor().fit(X_train, y_train)  
    print(clf.score(X_test, y_test))
    return clf

In [0]:
clf1 = xgb(good_Xmin1[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']], good_Xmin1.Xmin_true)
clf2 = xgb(good_Xmin2[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']], good_Xmin2.Ymin_true) 
clf3 = xgb(good_Xmin3[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']], good_Xmin3.Xmax_true)
clf4 = xgb(good_Xmin4[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']], good_Xmin4.Ymax_true)

0.6276761464768653
0.9832720210527273
0.9478375105683594
0.9613108718387856


...вот где собачка рылась!! ...ну-ну....



### Не буду выбирать "лучших усеров", а просто предскажу по медианным темам теста:

In [0]:
quo_test1 = votest.groupby("itemId")[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']].median().reset_index()
print(quo_test1.shape) 
quo_test1.head()

(630, 8)


,itemId,Xmin,L,H,P,S,L/H,D
0,18,135.0,594.0,367.0,1922.0,221404.0,1.825880,705.157780
1,19,28.0,85.0,165.0,500.0,14025.0,0.515152,185.607112
2,33,43.0,322.0,241.0,1126.0,77602.0,1.336100,402.200199
3,62,98.0,534.0,193.0,1320.0,93275.0,2.219512,542.210291
4,114,69.0,147.0,193.0,660.0,26901.0,0.780000,234.729632


In [0]:
quo_test1['ans1'] = clf1.predict(quo_test1[['Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']])
quo_test1.head()

,itemId,Xmin,L,H,P,S,L/H,D,ans1
0,18,135.0,594.0,367.0,1922.0,221404.0,1.825880,705.157780,118.283949
1,19,28.0,85.0,165.0,500.0,14025.0,0.515152,185.607112,44.819436
2,33,43.0,322.0,241.0,1126.0,77602.0,1.336100,402.200199,40.270964
3,62,98.0,534.0,193.0,1320.0,93275.0,2.219512,542.210291,108.891495
4,114,69.0,147.0,193.0,660.0,26901.0,0.780000,234.729632,62.901081


In [0]:
quo_test2 = votest.groupby("itemId")[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']].median().reset_index()
quo_test2['ans2'] = clf2.predict(quo_test2[['Ymin', 'L', 'H', 'P', 'S', 'L/H', 'D']])
print(quo_test2.shape) 
quo_test2.head()

quo_test3 = votest.groupby("itemId")[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']].median().reset_index()
quo_test3['ans3'] = clf3.predict(quo_test3[['Xmax', 'L', 'H', 'P', 'S', 'L/H', 'D']])
print(quo_test3.shape)
quo_test3.head()

quo_test4 = votest.groupby("itemId")[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']].median().reset_index()
quo_test4['ans4'] = clf4.predict(quo_test4[['Ymax', 'L', 'H', 'P', 'S', 'L/H', 'D']])
print(quo_test4.shape)
quo_test4.head()

(630, 9)
(630, 9)
(630, 9)


,itemId,Ymax,L,H,P,S,L/H,D,ans4
0,18,1071.0,594.0,367.0,1922.0,221404.0,1.825880,705.157780,1098.224497
1,19,727.0,85.0,165.0,500.0,14025.0,0.515152,185.607112,729.690519
2,33,629.0,322.0,241.0,1126.0,77602.0,1.336100,402.200199,635.715538
3,62,1163.0,534.0,193.0,1320.0,93275.0,2.219512,542.210291,1190.125428
4,114,763.0,147.0,193.0,660.0,26901.0,0.780000,234.729632,761.926880


In [0]:
# Пора объединяться....
quo_test1 = quo_test1.set_index('itemId')['ans1']
quo_test2 = quo_test2.set_index('itemId')['ans2']
quo_test3 = quo_test3.set_index('itemId')['ans3']
quo_test4 = quo_test4.set_index('itemId')['ans4']

quo_test = pd.concat([quo_test1, quo_test2, quo_test3, quo_test4], axis=1)
quo_test.head()

,ans1,ans2,ans3,ans4
itemId,,,,
18,118.283949,669.338047,762.714931,1098.224497
19,44.819436,560.124989,122.995691,729.690519
33,40.270964,386.675915,384.044554,635.715538
62,108.891495,945.547624,615.436764,1190.125428
114,62.901081,567.769083,218.182234,761.926880


In [0]:
quo_test.sort_index().to_csv("drive/My Drive/sna9/xgbregr_med.csv", header=None) # П: 0.5053647 предсказание! от медианы! а чего я хотел-то...